In [1]:
from bs4 import BeautifulSoup as bs
import os
import requests 
import pymongo
import pandas as pd
from splinter import Browser

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA MARS NEWS

In [4]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)


In [5]:
# pass the url into chrome broswer
html = browser.html
soup = bs(html, 'html.parser')
# grap the latest news
results = soup.select_one('li.slide')

In [6]:
# slice out the Title and Title Paragraph and store on variable title and title_para respectively
title = results.find('div', class_="content_title").text
title_para = results.find('div', class_="article_teaser_body").text

In [7]:
print("....the latest NASA news.....")
print("Title: " + title)
print("Title Paragraph: "+ title_para)

....the latest NASA news.....
Title: NASA 'Optometrists' Verify Mars 2020 Rover's 20/20 Vision
Title Paragraph: Mars 2020 rover underwent an eye exam after several cameras were installed on the rover.


# JPL Mars Space Images - Featured Image


In [8]:
# openning the url on chrome
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [9]:
full_image = browser.find_by_id('full_image')
full_image.click()


In [10]:
browser.is_element_present_by_text('more info', wait_time=1)
more_info = browser.find_link_by_partial_text('more info')
more_info.click()

In [11]:
html = browser.html
image_soup = bs(html, 'html.parser')

In [12]:
featured_image = image_soup.select_one('figure.lede a img').get("src")
featured_image

'/spaceimages/images/largesize/PIA12831_hires.jpg'

In [13]:
featured_image_url = f'https://www.jpl.nasa.gov{featured_image}'
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA12831_hires.jpg'

## Mars Weather Tweet

In [14]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [15]:
html = browser.html
soup_weather = bs(html, 'html.parser')

In [16]:
# scrape the first mars weather tweet
mars_tweet = soup_weather.select_one('div.js-tweet-text-container')

In [17]:
mars_weather = mars_tweet.find('p').text 
mars_weather

'InSight sol 245 (2019-08-05) low -99.9ºC (-147.8ºF) high -25.6ºC (-14.1ºF)\nwinds from the SSE at 4.6 m/s (10.2 mph) gusting to 17.7 m/s (39.5 mph)\npressure at 7.60 hPapic.twitter.com/kLqykpNHtR'

In [18]:
url = "http://space-facts.com/mars/"

In [19]:
mars_df = pd.read_html(url)
mars_data = mars_df[0]
mars_data

,Mars - Earth Comparison,Mars,Earth
0,Diameter:,"6,779 km","12,742 km"
1,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
2,Moons:,2,1
3,Distance from Sun:,"227,943,824 km","149,598,262 km"
4,Length of Year:,687 Earth days,365.24 days
5,Temperature:,-153 to 20 °C,-88 to 58°C


In [20]:
mars_fact = mars_data[["Mars - Earth Comparison", "Mars"]]

In [21]:
Mars_fact = mars_fact.rename(columns={"Mars - Earth Comparison":"Fact_parameters"})

In [22]:
Mars_fact.set_index('Fact_parameters', inplace = True)

In [23]:
Mars_fact

,Mars
Fact_parameters,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-153 to 20 °C


In [24]:
Mars_html = Mars_fact.to_html()

In [25]:
Mars_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Fact_parameters</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n  </tbody>\n</table>'

In [26]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)

In [27]:
html = browser.html
mars_hemi = bs(html, 'html.parser')

In [28]:
Results = mars_hemi.find_all('div', class_="item")

In [29]:
hemisphere_image_urls = []

# First, get a list of all of the hemispheres
links = browser.find_by_css("a.product-item h3")

# Next, loop through those links, click the link, find the sample anchor, return the href
for i in range(len(links)):
    hemisphere = {}
    
    # We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_css("a.product-item h3")[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.find_link_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']
    
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_css("h2.title").text
    
    # Append hemisphere object to list
    hemisphere_image_urls.append(hemisphere)
    
    # Finally, we navigate backwards
    browser.back()
    

In [30]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]